In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
from datetime import datetime

start_time = datetime.now()

    
url_site = 'https://www.tripadvisor.com'
url = 'https://www.tripadvisor.com/Restaurants-g294197-zfn15566165-Seoul.html'
driver = webdriver.Chrome('/Users/jonghyunkim/repo/Tristan/chromedriver')
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
pages = soup.select('#EATERY_LIST_CONTENTS > div.deckTools.btm > div > div > a')
pages_list = [url_site + page['href'] for page in pages]
pages_list = [url] + pages_list


restaurants_link_total = []
page_num = 1
for page in pages_list:
    if page_num != 1:
        driver.get(page)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
    print(f'Indexing pages : {page_num}/{len(pages_list)}          ', end='\r')
    restaurants = soup.select('#component_2 > div > div > span > div.bGnIM > div.emrzT.Vt.o > div.dzomp.u.F.f.Ff.K > div > span > a')
    restaurants_links = [url_site + restaurant['href'] for restaurant in restaurants]
    restaurants_link_total += restaurants_links
    page_num += 1
     
print('Indexing pages Done!                                  ')

context = []
for i, link in enumerate(restaurants_link_total):
    driver.get(link)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # name = soup.select('#component_32 > div > div.eTnlN._W.w.O > h1')[0].text
    name = soup.find(class_='eTnlN _W w O').text.replace(' UnclaimedSaveShare ','')
    review_num = soup.find(class_='eBTWs').text
    rating = soup.find(class_='RWYkj d H0')['aria-label'].replace(' of 5 bubbles','')
    address = soup.find_all(class_='fhGHT')[1].text
    try:
        phone = soup.find_all(class_='iPqaD _F G- ddFHE eKwUx')[1].text
    except:
        phone = 'na'

    restaurants_data = [name, review_num, rating, address, phone]
    context.append(restaurants_data)
    print(f'#{i+1}/{len(restaurants_link_total)} {name} done!                                 ', end='\r')

    
print('Cralwing pages Done!                                  ')
    
driver.quit()


df = pd.DataFrame(data=context, columns=['Name','# of Reviews','Review Rating','Address','Telephone'])
df.to_excel('kim_jiuk_2017120029.xlsx')

end_time = datetime.now()
print(f'Total restaurant : {i+1} crawled')
print(f'Total time : {end_time - start_time}')

df

Indexing pages Done!                                  
Cralwing pages Done!                                                                  
Total restaurant : 114 crawled
Total time : 0:04:30.138027


,Name,# of Reviews,Review Rating,Address,Telephone
0,Starbucks,16 reviews,4.5,"51, Inchon-ro 24-gil, Seongbuk-gu, Seoul 02856...",+82 2-758-8067
1,Goreun Sunshine,13 reviews,4.5,"27, Gaeunsa-Gil, Seoul South Korea",+82 2-953-3394
2,Samsung Whole Chicken,11 reviews,4.5,"60, Inchon-ro24-Gil, Seoul South Korea",+82 2-922-0077
3,Dongu Ox Bone Soup,9 reviews,4.5,"45, Gaeunsa-Gil, Seoul South Korea",na
4,Superb Flavour Ok,6 reviews,4.5,"22-6, Gaeunsa-Gil, Seoul South Korea",+82 2-929-7889
...,...,...,...,...,...
109,Seoul Ssam Cold Buckwheat Noodles,3 reviews,3.0,"21-4, Gaeunsa-Gil, Seoul South Korea",+82 2-929-4610
110,Fromaqi,0 reviews,-1.0,"13, Inchon-ro 24ga-gil, Seongbuk-gu, Seoul 028...",+82 70-4038-8537
111,Fromaqi,0 reviews,-1.0,"13, Inchon-ro 24ga-gil, Seongbuk-gu, Seoul 028...",+82 70-4038-8537
112,Sushi California Godae,0 reviews,-1.0,"120-55, Anamdong5-ga, Seongbuk-gu, Seoul South...",+82 2-924-0387
